In [33]:
! pip install transformers 
! pip install datasets 
! pip install huggingface_hub
! pip install googletrans==3.1.0a0

In [34]:
from datasets import load_dataset, load_metric

In [35]:
datasets=load_dataset("tydiqa", "secondary_task")

  0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
import transformers
print(transformers.__version__)

4.16.2


In [37]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 49881
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5077
    })
})

In [38]:
import copy
dstemp = copy.deepcopy(datasets)

In [39]:
dstemp

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 49881
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5077
    })
})

In [42]:
import re
def filter_dataset(ds,lang):
    def check(d):
        for i in lang:
            if re.search(i,d):
                return True
        return False
    ds = ds.filter(lambda x: check(x['id']))
    return ds

In [43]:
dstemp["train"]=filter_dataset(datasets["train"],['korean', 'indonesian', 'arabic', 'kiswahili', 'finnish', 'russian', 'english'])

In [44]:
dstemp

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 39173
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5077
    })
})

In [45]:
sd = dstemp.shuffle(seed=1234)

In [47]:
import googletrans
import pandas as pd

In [48]:
translator=googletrans.Translator()

In [49]:
trans_to_ben = translator.translate(sd['train']['question'][:20000],dest='bn')

In [50]:
df_to_ben = pd.DataFrame(trans_to_ben)

In [52]:
dict_to_ben={}
dict_to_ben['data']=[]
for i in range(len(df_to_ben)):
    dict_temp={}
    dict_temp['id']='bengali'+ '-1299177877317837031-' + str(i)
    dict_temp['context']=copy.deepcopy(sd['train'][i]['context'])
    dict_temp['answers']=copy.deepcopy(sd['train'][i]['answers'])
    dict_temp['title']=copy.deepcopy(sd['train'][i]['title'])
    dict_temp['question']=df_to_ben.iloc[i][0].text
    dict_to_ben['data'].append(dict_temp)

In [53]:
import json
with open("20k_bengali.json", "w") as outfile:
    json.dump(dict_to_ben, outfile)

In [71]:
trans_to_tel = translator.translate(sd['train']['question'][:20000],dest='telugu')

In [72]:
df_to_tel = pd.DataFrame(trans_to_tel)

In [73]:
dict_to_tel={}
dict_to_tel['data']=[]
for i in range(len(df_to_tel)):
    dict_temp={}
    dict_temp['id']='telugu'+ '-1299177877317837031-' + str(i)
    dict_temp['context']=copy.deepcopy(sd['train'][i]['context'])
    dict_temp['answers']=copy.deepcopy(sd['train'][i]['answers'])
    dict_temp['title']=copy.deepcopy(sd['train'][i]['title'])
    dict_temp['question']=df_to_tel.iloc[i][0].text
    dict_to_tel['data'].append(dict_temp)

In [74]:
import json
with open("20k_telegu.json", "w") as outfile:
    json.dump(dict_to_tel, outfile)

In [75]:
check_dataset_ben = load_dataset('json', data_files='20k_bengali.json', field='data')
check_dataset_tel = load_dataset('json', data_files='20k_telegu.json', field='data')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-c3ea38ada6e257c3/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [76]:
from datasets import concatenate_datasets
final_dataset = concatenate_datasets([check_dataset_ben['train'],check_dataset_tel['train']])

In [77]:
final_dataset

Dataset({
    features: ['id', 'context', 'answers', 'title', 'question'],
    num_rows: 40000
})

In [79]:
# from huggingface_hub import notebook_login
# notebook_login()

In [80]:
# final_dataset.push_to_hub("krinal214/tydiqa_ben_tel")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]